### 목표
- 네이버 영화리뷰 데이터 셋을 이용해 긍정리뷰 / 부정리뷰를 분류하는 모델을 만들어보자
- TF-IDF 방법을 적용해보자
- Konlpy 한국어 형태소 분석기를 설치하고 활용해보자
- 단어별 긍정/ 부정 정보를 시각화해서 확인해보자

In [1]:
# Java 버전 확인하기
# 자바 버전은 cmd창에서 java -version으로 확인할 수 있다.
# 1.7버전 이상 설치가 필요하다

In [2]:
# jpype1이라는 라이브러리는 java 라이브러리인 konlㅔy를 파이썬 환경에서 구동되도록 도와주는 라이브러리
!pip install jpype1

In [3]:
# konlpy 설치하기
# konlpy -> 한국어 형태소 분석기 모음집
!pip install konlpy

In [4]:
from konlpy.tag import Kkma

In [5]:
kkma = Kkma()

In [6]:
kkma.nouns('나비')

['나비']

### os에러가 뜨는 경우
- jvm.dll 파일을 찾을 수 없다는 에러가 뜰 것 
- 시스템 환경변수가 제대로 잡혔는지 확인을 해보자!

In [7]:
import os # 시스템 접근

In [8]:
print('JAVA_HOME'in os.environ)

False


In [9]:
# 파일 탐색기로 접근
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk1.8.0_202\bin'

In [10]:
print('JAVA_HOME'in os.environ)

True


### 네이버 데이터 로딩

In [11]:
import pandas as pd

In [12]:
df_train = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [13]:
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [14]:
# id : 평점과 리뷰를 적은 사용자의 번호
# document : 실제 리뷰 내용
# label : 긍정 / 부정, 1은 긍정 0은 부정

In [15]:
# 결측치 확인
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [16]:
# 결측치 확인
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


- train과 test에서 결측치 여부가 확인
- 각각 5개/3개의 결측치
- 결측치의 양이 많지 않음! , 실제 리뷰내용은 채워주기 어려움 -> 삭제

In [17]:
# 결측치 제거 -> 비어있는 값 삭제 함수 dropna
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [19]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


### 데이터 분할
- 문제와 정답 데이터로 분할

In [20]:
text_train = df_train['document']
y_train = df_train['label']
text_test = df_test['document']
y_test = df_test['label']

In [21]:
print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


**Vectorizer**
- Vectorizer라는 도구는 말 그대로 대상을 벡터라는 수치값으로 변화내주는 도구
- 기계학습 분야에서는 주로 텍스트 데이터를 쉽게 분석하기 위해 벡터화 시켜주는 경우가 많다.
- 숫자나 수치형의 벡터를 입력받기를 기대하는 머신러닝 모델을 실행하기 위해서 텍스트나 다른 형태의 데이터를 숫잔는 벡터로 나타낼 필요가 있다.

### BOW(단어 가방, Bag Of Word)
- 문서 또는 말뭉치를 벡터라는 수치값으로 변환하는 기본적인 방법
- 문서 내에 있는 모든 단어를 모아 하나의 가방으로 만들어서 특정 문서에 어떤 단어가 들어있는지 리스트 형태로 표현하고 순서-숫자의 형태로 나타내는 방법

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
text_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [24]:
# Tf_idf 벡터라이저 객체 생성
tf_idf_vect = TfidfVectorizer()
# 단어가방 만들기
tf_idf_vect.fit(text_train[:3])
# 분리된 문장으로 만든 단어사전 출력
tf_idf_vect.vocabulary_.keys()

dict_keys(['더빙', '진짜', '짜증나네요', '목소리', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나', '너무재밓었다그래서보는것을추천한다'])

- 기본 TF-IDF 벡터라이저는 기본적으로 어절 단위로 끊어준다 -> 한국어 특성상 의미 파악이 어렵다
    - 어절 (문장을 이루는 토막)
- 조금 더 원활한 분석을 위해 의미를 가진 더 작은 단위로 분할해주자(형태소 단위)

### 한국어 형태소 분석기
**konlpy-Kkma** 꼬꼬마, 서울대학교 IDS(Intelligent Data System) 연구실 개발
- nouns() : 명사 추출
- morphs() : 형태소 추출
- pos() : 형태소 추출 + 품사 부착

In [25]:
# Mecab를 제외한 나머지는 구동이 잘됨
from konlpy.tag import Kkma

In [26]:
kkma=Kkma()

In [27]:
text_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [28]:
# 명사 추출
kkma.nouns(text_train[0])

['더빙', '목소리']

In [29]:
# TF-IDF와 kkma.nouns() 연결해서 한국어 명사 추출기를 만들어보자
def myTokenizer(text) : 
    return kkma.nouns(text)

In [30]:
tmp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer)
tmp_tf_idf.fit(text_train[:3])
tmp_tf_idf.vocabulary_.keys()

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


dict_keys(['더빙', '목소리', '흠', '포스터', '포스터보고', '보고', '초', '초딩영화줄', '딩', '영화', '줄', '오버', '오버연기', '연기', '재', '재밓', '밓', '추천'])

### pos tagging 활용
- 감성 분석 => 형용사랑 동사를 주로 사용!

pos tagging = 형태소를 추출해서 품사를 태깅해주는 기능

In [31]:
# 품사 태그의 종류를 확인해보자
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [32]:
# 단순 형태소만 추출
data = '집 가다 가고싶다 배고프다 먹다 모자 안경'
kkma.morphs(data)
# => 단순 형태소 별로 분할

['집', '가다', '가', '고', '싶', '다', '배고프', '다', '먹', '다', '모자', '안경']

In [33]:
# 형태소 추출 및 품사 태깅
kkma.pos(data)
# 여기서 형용사와 동사만 빼려면??

[('집', 'NNG'),
 ('가다', 'MAG'),
 ('가', 'VV'),
 ('고', 'ECE'),
 ('싶', 'VXA'),
 ('다', 'ECS'),
 ('배고프', 'VA'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('모자', 'NNG'),
 ('안경', 'NNG')]

In [34]:
d= pd.DataFrame(kkma.pos(data), columns=['morphs', 'tag'])
d.set_index('tag', inplace=True)
# 명사, 형용사, 동사만 추출해보자!
d.loc[d.index.intersection(['VV','VA','NNG'])]

,morphs
tag,
NNG,집
NNG,모자
NNG,안경
VV,가
VV,먹
VA,배고프


In [35]:
# text_train[:3] 실전 데이터 적용해보기
def myTokenizer2(text):
    d= pd.DataFrame(kkma.pos(text), columns=['morphs', 'tag'])
    d.set_index('tag', inplace=True)
    # 명사, 형용사, 동사만 추출해보자!
    if('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index) :
        # 명사 형용사 동사만 추출
        return d.loc[d.index.intersection(['VV','VA','NNG']), 'morphs'].values
    else : 
        return[]

In [36]:
tmp_tf_idf = TfidfVectorizer(tokenizer= myTokenizer2)
tmp_tf_idf.fit(text_train[:3])
tmp_tf_idf.vocabulary_

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [37]:
final_tf_idf = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf.fit(text_train[:10000])

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x0000020B6700D670>)

In [39]:
# 구축한 단어사전 확인해보기
print(f"구축한 단어사전 개수 : {len(final_tf_idf.vocabulary_)}")

구축한 단어사전 개수 : 8748


In [40]:
# 벡터화 진행!
X_train = final_tf_idf.transform(text_train[:10000])
X_test = final_tf_idf.transform(text_test[:10000])

### 모델링
- 텍스트마이닝은 시간이 많이 걸리는 작업이다.  
  그래서 상대적으로 속도가 빠른 로지스틱 회귀 모델을 사용해서 빠른 분석 결과를 얻어보자.


In [41]:
# 모델 import 
from sklearn.linear_model import LogisticRegression # 선형 분류/ 로지스틱 회귀

In [42]:
# 모델 객체 생성
logi_model = LogisticRegression()

In [44]:
# 모델 학습
logi_model.fit(X_train, y_train[:10000])

LogisticRegression()

In [45]:
# 스코어 확인
logi_model.score(X_test, y_test[:10000])

0.7843

In [46]:
# 위에 학습된 모델을 활용
# predict_proba : 분류의 불확실성
target_name = ['부정', '긍정']
review = ['오랜만에 볼만한 한국 영화였어요.']
vect_review = final_tf_idf.transform(review)
pre = logi_model.predict(vect_review)
print(f"{review[0]}는 {logi_model.predict_proba(vect_review).max()*100:.2f}%로 {target_name[pre[0]]}리뷰입니다.")

오랜만에 볼만한 한국 영화였어요.는 81.49%로 긍정리뷰입니다.


In [47]:
review = ['정말 재밌는 영화네요!.']
vect_review = final_tf_idf.transform(review)
pre = logi_model.predict(vect_review)
print(f"{review[0]}는 {logi_model.predict_proba(vect_review).max()*100:.2f}%로 {target_name[pre[0]]}리뷰입니다.")

정말 재밌는 영화네요!.는 98.41%로 긍정리뷰입니다.


In [48]:
review = ['개노잼']
vect_review = final_tf_idf.transform(review)
pre = logi_model.predict(vect_review)
print(f"{review[0]}는 {logi_model.predict_proba(vect_review).max()*100:.2f}%로 {target_name[pre[0]]}리뷰입니다.")

개노잼는 71.31%로 부정리뷰입니다.


단어사전 확인해보기

In [49]:
final_tf_idf.vocabulary_

{'아': 4671,
 '짜증나': 7146,
 '더빙': 1742,
 '목소리': 2611,
 '흠': 8699,
 '포스터': 8014,
 '보고': 3208,
 '영화': 5271,
 '줄': 6884,
 '오버': 5359,
 '연기': 5172,
 '가볍': 55,
 '재': 6338,
 '추천': 7428,
 '어': 4938,
 '보': 3205,
 '교도소': 693,
 '이야기': 5907,
 '재미': 6353,
 '점': 6532,
 '조정': 6728,
 '멀': 2477,
 '없': 5041,
 '익살': 5974,
 '스파이': 4371,
 '더': 1730,
 '맨': 2450,
 '커스틴': 7565,
 '돋보이': 1829,
 '늙': 1489,
 '보이': 3238,
 '하': 8148,
 '이쁘': 5879,
 '걸음마': 353,
 '초등학교': 7354,
 '학년': 8209,
 '생인': 3875,
 '용': 5522,
 '별': 3186,
 '반개': 2939,
 '떼': 2070,
 '아깝': 4677,
 '원작': 5630,
 '긴장감': 1044,
 '살리': 3740,
 '욕': 5513,
 '이응': 5925,
 '경': 417,
 '길': 1047,
 '우': 5538,
 '생활': 3882,
 '발': 2969,
 '납': 1281,
 '감금': 145,
 '반복': 2947,
 '이': 5818,
 '드라마': 1947,
 '가족': 92,
 '사람': 3649,
 '모': 2550,
 '엿': 5233,
 '나오': 1205,
 '낫': 1284,
 '액션': 4881,
 '안': 4775,
 '있': 6141,
 '평점': 7994,
 '우드': 5541,
 '식': 4487,
 '화려': 8540,
 '낮': 1289,
 '헐': 8403,
 '길들이': 1048,
 '인피': 6036,
 '니트': 1507,
 '짱': 7158,
 '나': 1168,
 '죽': 6869,
 '때': 2042,
 '

In [51]:
voc = pd.DataFrame(final_tf_idf.vocabulary_.keys(),
                    index=final_tf_idf.vocabulary_.values())
voc.sort_index()

,0
0,ㄱ
1,ㄴ
2,ㄷ
3,ㄹ
4,ㅁ
...,...
8743,힘쓰
8744,힘없
8745,힘입
8746,힙합


### 가중치 확인해보기

In [53]:
learning_rs = pd.DataFrame(logi_model.coef_.T, index = voc.sort_index()[0], columns=['w'])
learning_rs.sort_values(by='w')

,w
0,
재미없,-4.478074
최악,-4.288826
아깝,-4.098801
쓰레기,-4.057784
실망,-3.632922
...,...
수작,3.304806
좋,3.326942
재미있,3.726432


In [54]:
from sklearn.linear_model import LinearRegression
LinearRegression()